## Running the metrics for proof generation

In [1]:
import sys
sys.path.append("..")

In [2]:
import json
import transformers
import torch
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import glob
from pprint import pprint
import transformers
import os
import datasets
datasets.logging.set_verbosity(50)
transformers.logging.set_verbosity_error()

%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Data

In [3]:
ds_base = json.load(open(
    '../data/base/proofwiki.json'
))
ds_generations = json.load(open(
    '../data/base/proofwiki__refs_ground_truth.json'
))
redirects = json.load(open(
    '../data/base/proofwiki_redirects.json'
))

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained('facebook/bart-large')

### Utilities

In [5]:
def get_theorem_ids(split):
    import csv
    with open('../data/base/core_evalset.tsv') as f:
        reader = csv.DictReader(f, delimiter='\t')
        core_evalset = [row for row in reader]
        theorem_ids = [int(row['theorem_id']) for row in core_evalset if split == 'both' or row['split'] == split]
    return theorem_ids

def load_gens(name2gen_files, split=None):
    name2gens = {}
    for name in name2gen_files:
        try:
            name2gens[name] = json.load(open(name2gen_files[name]))
        except:
            try:
                name2gens[name] = pickle.load(open(name2gen_files[name], 'rb'))
            except:
                print("skipping %s" % name)
        if split is not None:
            theorem_ids = get_theorem_ids(split)
            name2gens[name]['full_generations'] = [x for x in name2gens[name]['full_generations'] if x['metadata'][0] in theorem_ids]
    return name2gens

In [6]:
from npgen.evaluation_proofgen import full_generation_metrics

def evaluate(name2gens):    
    name2metrics_object = {}
    df_data = []
    for name, gens in tqdm(name2gens.items(), total=len(name2gens)):
        metrics, metrics_object = full_generation_metrics(
            gens['full_generations'], 
            ds_base, 
            ds_generations,
            redirects,
            tokenizer, 
            return_metrics_object=True
        )
        name, run = name.split('___')
        for span in name.split('__'):
            if '=' in span:
                key, value = span.split('=')
                metrics[key] = value
        metrics['name'] = name
        metrics['run'] = run
        df_data.append(metrics)
        name2metrics_object[name] = metrics_object

    df = pd.DataFrame(df_data)
    return df

In [7]:
from copy import deepcopy

default_cols = ['ppl', 'gleu', 'ref_f1', 'kf1', 'f1', 'corpus_ref_halluc']
def display(df, cols=default_cols, sort=None):
    df = deepcopy(df)  
    if cols is None:
        min_cols = ['corpus_ref_halluc', 'ppl']
        df_show = df[['run', 'ppl', 'name', 'f1', 'gleu', 'ref_precision', 'ref_recall', 'ref_f1', 'corpus_ref_halluc']]    
    else:
        min_cols = [c for c in ['corpus_ref_halluc', 'ppl'] if c in cols]
        show_cols = cols
        df_show = df[['run', 'name'] + show_cols]   

    df_show = df_show.groupby('name').mean().round(4)
    df_show = df_show.round(2)
    if sort is not None:
        df_show = df_show.sort_values(sort)

    return df_show

### GPT-3

In [ ]:
split = 'test'

name2gen_files = {    
    'gpt3___1': '../other/naturalprover_generations/gpt3.pkl',
    'naturalprover-retrieve___1': '../other/naturalprover_generations/naturalprover_retrieved.pkl',
    'naturalprover___1': '../other/naturalprover_generations/naturalprover.pkl',    
    'naturalprover++___1': '../other/naturalprover_generations/naturalprover_plusplus.pkl',
}
name2gen = load_gens(name2gen_files, split=split)
df = evaluate(name2gen)

In [9]:
display(df, cols=['gleu', 'f1', 'kf1', 'ref_precision', 'ref_recall', 'ref_f1', 'corpus_ref_halluc'], sort='ref_f1')

,gleu,f1,kf1,ref_precision,ref_recall,ref_f1,corpus_ref_halluc
name,,,,,,,
gpt3,24.40,49.96,49.30,29.93,24.73,23.69,17.92
naturalprover-retrieve,26.58,53.02,55.88,38.17,28.48,27.10,2.25
naturalprover,35.27,66.00,90.07,93.05,86.05,87.08,1.60
naturalprover++,34.49,65.61,96.39,94.66,95.00,93.92,1.71


### GPT-J/2

In [ ]:
split='valid'
name2gen_files = {    
    'gpt3curie-greedy___1': '../other/naturalprover_generations/naturalprover.pkl',
    'gpt2-greedy___1': '../other/naturalprover_generations/naturalprover_gpt2.json',
    'gptj6b-greedy___1': '../other/naturalprover_generations/naturalprover_gptj6b.json',
    'gptj6b-select10___1': '../other/naturalprover_generations/naturalprover_gptj6b_select10.json',
}
name2gen = load_gens(name2gen_files, split=split)
df = evaluate(name2gen)

In [11]:
display(df, cols=['gleu', 'ref_f1', 'corpus_ref_halluc'], sort='ref_f1')

,gleu,ref_f1,corpus_ref_halluc
name,,,
gpt2-greedy,32.06,65.22,6.76
gptj6b-greedy,38.58,79.19,2.96
gptj6b-select10,37.83,88.80,4.84
gpt3curie-greedy,42.39,89.29,1.90


### Example theorem and generated proof 

(GPTJ-6B with sample-and-select; constraint value function; 10 samples)

In [15]:
import json
data = json.load(open('../other/naturalprover_generations/naturalprover_gptj6b_select10.json'))

index = 2565

proofwiki = json.load(open('../data/base/proofwiki.json'))
theorem = proofwiki['dataset']['theorems'][index]
generated_proof = [proof for proof in data['full_generations'] if proof['metadata'][0] == 2565][0]

text = '\n\n'.join([theorem['title'], '\n'.join(theorem['contents']), generated_proof['y']]).replace('\\n', '\n')

print(text)

Integers under Multiplication form Countably Infinite Semigroup

The [[Definition:Set|set]] of [[Definition:Integer|integers]] under [[Definition:Integer Multiplication|multiplication]] $\struct {\Z, \times}$ is a [[Definition:Countably Infinite Set|countably infinite]] [[Definition:Semigroup|semigroup]].

From [[Integers under Multiplication form Semigroup]], $\struct {\Z, \times}$ is a [[Definition:Semigroup|semigroup]].
From [[Integers are Countably Infinite]], $\struct {\Z, \times}$ is [[Definition:Countably Infinite Set|countably infinite]].
{{qed}}
